<a href="https://colab.research.google.com/github/jpbeaud/accident/blob/main/passagers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Charger les données du Titanic depuis un fichier CSV
url = "https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153328/usagers-2023.csv"
data = pd.read_csv(url, delimiter=";")
print("Colonnes=", data.columns)
print("Données=", data.head())
print("Taille=", data.shape)


Colonnes= Index(['Num_Acc', 'id_usager', 'id_vehicule', 'num_veh', 'place', 'catu',
       'grav', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2', 'secu3', 'locp',
       'actp', 'etatp'],
      dtype='object')
Données=         Num_Acc    id_usager  id_vehicule num_veh  place  catu  grav  sexe  \
0  202300000001  203 851 184  155 680 557     A01      1     1     4     1   
1  202300000002  203 851 182  155 680 556     A01      1     1     1     2   
2  202300000002  203 851 183  155 680 556     A01     10     3     3     1   
3  202300000003  203 851 180  155 680 554     B01      1     1     3     1   
4  202300000003  203 851 181  155 680 555     A01      1     1     1     2   

   an_nais  trajet  secu1  secu2  secu3  locp actp  etatp  
0   1978.0       5      2      0     -1    -1   -1     -1  
1   1997.0       9      1      0     -1    -1   -1     -1  
2   1997.0       9      0     -1     -1     2    3      1  
3   1987.0       0      2      6      0     0    0     -1  
4   1984.0  

Suppression des colonnes non utiles

In [12]:
data.drop(['Num_Acc', 'id_usager', 'id_vehicule', 'num_veh', 'locp', 'actp', 'etatp'], axis=1, inplace=True)
print("Taille=", data.shape)
print("Colonnes=", data.columns)


Taille= (125789, 9)
Colonnes= Index(['place', 'catu', 'grav', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2',
       'secu3'],
      dtype='object')


In [13]:
print(data.head())

   place  catu  grav  sexe  an_nais  trajet  secu1  secu2  secu3
0      1     1     4     1   1978.0       5      2      0     -1
1      1     1     1     2   1997.0       9      1      0     -1
2     10     3     3     1   1997.0       9      0     -1     -1
3      1     1     3     1   1987.0       0      2      6      0
4      1     1     1     2   1984.0       0      1      0      0


In [15]:
# prompt: dire les types de données de data

data.dtypes

,0
place,int64
catu,int64
grav,int64
sexe,int64
an_nais,float64
trajet,int64
secu1,int64
secu2,int64
secu3,int64


In [20]:

# Sélectionner les caractéristiques et la cible
features = ['place', 'catu', 'sexe', 'an_nais', 'trajet', 'secu1', 'secu2','secu3']
target = 'grav'

# Prétraiter les données
#data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})
data['an_nais'].fillna(data['an_nais'].mean(), inplace=True)


<ipython-input-20-fe5b1bcaaaf0>:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['an_nais'].fillna(data['an_nais'].mean(), inplace=True)


C

In [22]:

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(data[features], data[target], test_size=0.2, random_state=42)

# Créer et entraîner le modèle de régression logistique
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Prédire les résultats sur l'ensemble de test
y_pred = model.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle: {accuracy:.2f}")




Précision du modèle: 0.53


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Exemple de prédiction

In [19]:
# Exemple de prédiction pour un passager fictif
passager_fictif = pd.DataFrame({
    'place':[2], 'catu':[2], 'sexe':[1], 'an_nais':[1978.0], 'trajet':[5], 'secu1':[2], 'secu2':[0],'secu3':[-1]
})

prediction = model.predict(passager_fictif)
print(f"Le passager fictif a {'survécu' if prediction[0] == 1 else 'péri'}")

ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- grav


autre modèle prédictif

In [28]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

model2 = DecisionTreeClassifier()
model2.fit(X_train, y_train)
# Prédire les résultats sur l'ensemble de test
y_pred = model2.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle: {accuracy:.2f}")

# Définir la grille d'hyperparamètres
#param_grid = { 'criterion': ['gini', 'entropy'], 'max_depth': [None, 10, 20, 30, 40], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4] }
# Configurer le GridSearchCV
#grid_search = GridSearchCV(estimator=model2, param_grid=param_grid, cv=5, scoring='accuracy')
# Entraîner le modèle avec Grid Search
#grid_search.fit(X_train, y_train)
# Afficher les meilleurs hyperparamètres
#print(f'Best hyperparameters: {grid_search.best_params_}')

#'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10
model3 = DecisionTreeClassifier(criterion='entropy', max_depth=10, min_samples_leaf=1, min_samples_split=10)
model3.fit(X_train, y_train)
# Prédire les résultats sur l'ensemble de test
y_pred = model3.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f"Précision du modèle: {accuracy:.2f}")


Précision du modèle: 0.56
Précision du modèle: 0.59


In [29]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from scipy.stats import randint

# Définir le modèle
model = DecisionTreeClassifier()

# Définir la distribution des hyperparamètres
param_dist = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30, 40],
    'min_samples_split': randint(2, 11),
    'min_samples_leaf': randint(1, 5)
}

# Configurer le RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)

# Entraîner le modèle avec Random Search
random_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print(f'Best hyperparameters: {random_search.best_params_}')


Best hyperparameters: {'criterion': 'entropy', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10}


méthode bayésien pour optimiser

In [32]:
!pip install skopt
from skopt import BayesSearchCV
from sklearn.tree import DecisionTreeClassifier

# Définir le modèle
model = DecisionTreeClassifier()

# Définir la recherche bayésienne
bayes_search = BayesSearchCV(
    estimator=model,
    search_spaces={
        'criterion': ['gini', 'entropy'],
        'max_depth': (1, 40),
        'min_samples_split': (2, 10),
        'min_samples_leaf': (1, 4)
    },
    n_iter=32,
    cv=5,
    scoring='accuracy'
)

# Entraîner le modèle avec la recherche bayésienne
bayes_search.fit(X_train, y_train)

# Afficher les meilleurs hyperparamètres
print(f'Best hyperparameters: {bayes_search.best_params_}')


ERROR: Could not find a version that satisfies the requirement skopt (from versions: none)
ERROR: No matching distribution found for skopt


ModuleNotFoundError: No module named 'skopt'